In [11]:
# 導入所需套件
import pandas as pd                 # 數據處理和分析
import numpy as np                  # 數值運算
import matplotlib.pyplot as plt     # 基礎繪圖
import seaborn as sns               # 進階繪圖，基於 matplotlib
from scipy import stats             # 用於統計檢定
import warnings                     # 用於處理警告訊息

In [12]:
# 忽略警告訊息，讓 Notebook 輸出更簡潔
warnings.filterwarnings('ignore')

# 設定 Matplotlib 顯示中文 (如果你的系統需要)
plt.rcParams['font.sans-serif'] = ['Microsoft YaHei', 'SimHei'] # 替換為你的中文字體
plt.rcParams['axes.unicode_minus'] = False # 解決負號顯示問題

In [13]:
# 定義數據檔案路徑
data_path = '../data/train.csv' # data path

# 載入數據集
try:
    df = pd.read_csv(data_path)
    print("數據載入成功！")
    print(f"數據集包含 {df.shape[0]} 行和 {df.shape[1]} 列。")
except FileNotFoundError:
    print(f"錯誤：找不到數據檔案。請確認 '{data_path}' 路徑正確。")
    print("請將 'train.csv' 放到 projects/EDA_Titanic_Survival_Analysis/data/ 資料夾中。")

數據載入成功！
數據集包含 891 行和 12 列。


In [14]:
# 顯示前5行數據
print("數據集前5行：")
print(df.head())

# 顯示數據集的基本信息 (欄位、非空值數量、數據類型)
print("\n數據集基本信息：")
df.info()

# 顯示數據集的描述性統計 (針對數值型欄位)
print("\n數據集描述性統計：")
print(df.describe())

數據集前5行：
   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN      

### 資料清洗與前處理

#### 缺失值處裡

In [15]:
print("--- 檢查缺失值 ---")
print(df.isnull().sum()) # 顯示每個欄位的缺失值數量
print("\n缺失值比例 (%):")
print((df.isnull().sum() / len(df) * 100).sort_values(ascending=False)) # 顯示缺失值比例，並按缺失數量降序排列

--- 檢查缺失值 ---
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

缺失值比例 (%):
Cabin          77.104377
Age            19.865320
Embarked        0.224467
PassengerId     0.000000
Name            0.000000
Pclass          0.000000
Survived        0.000000
Sex             0.000000
Parch           0.000000
SibSp           0.000000
Fare            0.000000
Ticket          0.000000
dtype: float64


In [16]:
print("\n--- 處理 Cabin 欄位（缺失率過高，新增一布林值變數代表[有無客艙號資料]以取代該欄位） ---")
# 創建一個新的特徵 'Has_Cabin'：1 = 有客艙號，0 = 沒有客艙號
df['Has_Cabin'] = df['Cabin'].apply(lambda x: 0 if pd.isna(x) else 1)

print(df[['Cabin']].head()) # 檢查新欄位（此行會報錯因為Cabin已丟棄，可刪除或註釋掉）
# 丟棄原始的 Cabin 欄位
df = df.drop('Cabin', axis=1)
print("已創建 'Has_Cabin' 欄位並丟棄原始 'Cabin' 欄位。")
print(df[['Has_Cabin']].head()) # 檢查新欄位（此行會報錯因為Cabin已丟棄，可刪除或註釋掉）
print(df['Has_Cabin'].value_counts())


--- 處理 Cabin 欄位（缺失率過高，新增一布林值變數代表[有無客艙號資料]以取代該欄位） ---
  Cabin
0   NaN
1   C85
2   NaN
3  C123
4   NaN
已創建 'Has_Cabin' 欄位並丟棄原始 'Cabin' 欄位。
   Has_Cabin
0          0
1          1
2          0
3          1
4          0
Has_Cabin
0    687
1    204
Name: count, dtype: int64
